In [ ]:
from utils_demo import *
from loguru import logger
from unittest.mock import patch
import warnings
warnings.filterwarnings('ignore')
from app_decomposer import DEFAULT_CONFIGURATION, API_DICT_TS, IOI_SAMPLING_PERIOD, PERF_MODEL_DATASET_NAME

In [ ]:
fig = plot_detected_phases(jobid=5170, merge=True, show_phases=True, width=1200, height=600)
fig.show() 

In [ ]:
%%capture
logger.remove()
def decompose_ioi_job(jobid):
    with patch.object(ComplexDecomposer, 'get_job_timeseries') as mock_get_timeseries:
        with patch.object(Configuration, 'get_kc_token') as mock_get_kc_token:
            with patch.object(ComplexDecomposer, 'get_job_node_count') as mock_get_node_count:
                timeseries = get_job_timeseries_from_file(job_id=jobid)                
                mock_get_timeseries.return_value = timeseries
                mock_get_kc_token.return_value = 'token'
                mock_get_node_count.return_value = 1
                # init the job decomposer
                cd = ComplexDecomposer(v0_threshold=0.02)
                return cd
        
# Launch decomposition on the signal
cd = decompose_ioi_job(jobid=5170)

representation = cd.get_job_representation(merge_clusters=True)
print(representation)
# This is the app encoding representation for Execution Simulator
print(f"compute={representation['events']}, reads={representation['read_volumes']}" 
      f"read_bw={representation['read_bw']}, writes={representation['write_volumes']}"
      f"write_bw={representation['write_bw']}, read_pattern={representation['read_pattern']}")
# Normalize signals to seconds and MB
timestamps = (cd.timestamps - cd.timestamps[0])/5
original_read = cd.read_signal/1e6
original_write = cd.write_signal/1e6

In [ ]:
# Apply same BW as measured by IOI
read_bw = list(map(lambda x: x/1e6, representation['read_bw']))
write_bw = list(map(lambda x: x/1e6, representation['write_bw']))
io_bw = list(map(lambda x, y: x + y, read_bw, write_bw))

In [ ]:
sim_time, sim_read_bw, sim_write_bw = simulate_app(representation['events'],
                                                   representation['read_volumes'],
                                                   representation['write_volumes'], io_bw, app_name="job")

In [ ]:
fig = display_original_sim_signals((sim_time, sim_read_bw, sim_write_bw),
                                   (timestamps, original_read, original_write),
                                   width=1400, height=900)
fig.show()